In [8]:
# ======================================================================
# STAGE-1: Preprocess ALL discharge files from nested folders
# ======================================================================

import os
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm


# -----------------------------------------------------------
# CONFIG
# -----------------------------------------------------------
EXTRACTED_FOLDER = Path("/content/drive/MyDrive/4 month project/DataSet/Extracted Files/")
OUTPUT_FOLDER = Path("/content/drive/MyDrive/4 month project/DataSet/Preprocessed Files/")
OUTPUT_FOLDER.mkdir(exist_ok=True)

MASTER_OUTPUT = OUTPUT_FOLDER / "all_batteries_preprocessed.csv"

# -----------------------------------------------------------
# Function: Extract Bxxxx from filename like "B0025_discharge.csv"
# -----------------------------------------------------------
import re

def get_battery_id(fname: str):
    """Extract Bxxxx from patterns like B0005_discharge.csv"""
    m = re.search(r"([Bb]\d{4})", fname)
    return m.group(1).upper() if m else None


# -----------------------------------------------------------
# 1. Collect ONLY discharge CSV files from ALL subfolders
# -----------------------------------------------------------
print("Scanning folders for *discharge.csv files* ...")

discharge_files = []
for f in EXTRACTED_FOLDER.rglob("*_discharge.csv"):
    discharge_files.append(f)

print(f"Found {len(discharge_files)} discharge files.")


# -----------------------------------------------------------
# 2. Process each discharge file → cycle-level data
# using your already-corrected functions:
#    - load_and_clean_discharge()
#    - aggregate_cycles_from_df()
#    - compute_soh_and_rul()
#    - process_single_battery_file()
# -----------------------------------------------------------

battery_groups = {}

for file in tqdm(discharge_files, desc="Processing discharge files"):
    bid = get_battery_id(file.name)
    if bid is None:
        print(f"⚠️ Could not detect battery ID: {file.name}")
        continue

    try:
        df_cycles = process_single_battery_file(file)
        df_cycles["battery_id"] = bid

        if bid not in battery_groups:
            battery_groups[bid] = []
        battery_groups[bid].append(df_cycles)

    except Exception as e:
        print(f"❌ Error processing file {file.name}: {e}")


# -----------------------------------------------------------
# 3. Merge repeated batteries correctly
# -----------------------------------------------------------
final_battery_dfs = {}

for bid, dfs in battery_groups.items():
    print(f"Merging battery {bid} with {len(dfs)} segments...")

    merged = pd.concat(dfs, ignore_index=True)

    # Sort cycles properly
    merged = merged.sort_values("cycle_no")

    # Remove duplicates (same cycle from multiple logs)
    merged = merged.drop_duplicates("cycle_no")

    # After merging → recompute SOH & RUL for the full battery life
    merged = compute_soh_and_rul(merged)

    final_battery_dfs[bid] = merged

    # Save per-battery file
    out_path = OUTPUT_FOLDER / f"{bid}_processed.csv"
    merged.to_csv(out_path, index=False)


# -----------------------------------------------------------
# 4. Create global master file
# -----------------------------------------------------------
master_df = pd.concat(final_battery_dfs.values(), ignore_index=True)
master_df = master_df.sort_values(["battery_id", "cycle_no"])
master_df.to_csv(MASTER_OUTPUT, index=False)

print("\n==============================================================")
print("STAGE-1 DISCHARGE PREPROCESSING COMPLETE")
print("Saved per-battery cleaned files in:", OUTPUT_FOLDER)
print("Saved master file:", MASTER_OUTPUT)
print("==============================================================")
print(master_df.head())


Scanning folders for *discharge.csv files* ...
Found 38 discharge files.


Processing discharge files:   3%|▎         | 1/38 [00:00<00:31,  1.17it/s]

DEBUG: Original columns in B0028_discharge.csv: ['Group_Name', 'Battery_ID', 'Cycle_No', 'Ambient_Temperature', 'Time', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Capacity']
DEBUG: Resolved canonical to actual column mapping: {'time': 'Time', 'voltage': 'Voltage_measured', 'current': 'Current_measured', 'temperature': 'Temperature_measured', 'capacity': 'Capacity', 'cycle_no': 'Cycle_No'}
DEBUG: Original columns in B0027_discharge.csv: ['Group_Name', 'Battery_ID', 'Cycle_No', 'Ambient_Temperature', 'Time', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Capacity']
DEBUG: Resolved canonical to actual column mapping: {'time': 'Time', 'voltage': 'Voltage_measured', 'current': 'Current_measured', 'temperature': 'Temperature_measured', 'capacity': 'Capacity', 'cycle_no': 'Cycle_No'}


Processing discharge files:   8%|▊         | 3/38 [00:03<00:35,  1.01s/it]

DEBUG: Original columns in B0026_discharge.csv: ['Group_Name', 'Battery_ID', 'Cycle_No', 'Ambient_Temperature', 'Time', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Capacity']
DEBUG: Resolved canonical to actual column mapping: {'time': 'Time', 'voltage': 'Voltage_measured', 'current': 'Current_measured', 'temperature': 'Temperature_measured', 'capacity': 'Capacity', 'cycle_no': 'Cycle_No'}


...

Processing discharge files:  97%|█████████▋| 37/38 [00:34<00:00,  1.15it/s]

DEBUG: Original columns in B0050_discharge.csv: ['Group_Name', 'Battery_ID', 'Cycle_No', 'Ambient_Temperature', 'Time', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Capacity']
DEBUG: Resolved canonical to actual column mapping: {'time': 'Time', 'voltage': 'Voltage_measured', 'current': 'Current_measured', 'temperature': 'Temperature_measured', 'capacity': 'Capacity', 'cycle_no': 'Cycle_No'}


Processing discharge files: 100%|██████████| 38/38 [00:34<00:00,  1.09it/s]

DEBUG: Original columns in B0049_discharge.csv: ['Group_Name', 'Battery_ID', 'Cycle_No', 'Ambient_Temperature', 'Time', 'Voltage_measured', 'Current_measured', 'Temperature_measured', 'Capacity']
DEBUG: Resolved canonical to actual column mapping: {'time': 'Time', 'voltage': 'Voltage_measured', 'current': 'Current_measured', 'temperature': 'Temperature_measured', 'capacity': 'Capacity', 'cycle_no': 'Cycle_No'}
Merging battery B0028 with 2 segments...
Merging battery B0027 with 2 segments...
Merging battery B0026 with 2 segments...
Merging battery B0025 with 2 segments...


Merging battery B0005 with 1 segments...
Merging battery B0007 with 1 segments...
Merging battery B0018 with 1 segments...
Merging battery B0006 with 1 segments...
Merging battery B0046 with 1 segments...
Merging battery B0045 with 1 segments...
Merging battery B0047 with 1 segments...
Merging battery B0048 with 1 segments...
Merging battery B0031 with 1 segments...
Merging battery B0032 with 1 segments...
Merging battery B0030 with 1 segments...
Merging battery B0029 with 1 segments...
Merging battery B0034 with 1 segments...
Merging battery B0033 with 1 segments...
Merging battery B0038 with 1 segments...
Merging battery B0036 with 1 segments...
Merging battery B0039 with 1 segments...
Merging battery B0041 with 1 segments...
Merging battery B0040 with 1 segments...
Merging battery B0042 with 1 segments...
Merging battery B0043 with 1 segments...
Merging battery B0044 with 1 segments...
Merging battery B0053 with 1 segments...
Merging battery B0054 with 1 segments...
Merging battery 

In [10]:
len(final_battery_dfs)

34

In [11]:
for bid in list(final_battery_dfs.keys())[:1]:
    print("Battery:", bid)
    display(final_battery_dfs[bid].head(10))

Battery: B0028


,cycle_no,time_start,time_end,discharge_time_sec,voltage_min,voltage_max,voltage_mean,voltage_std,current_min,current_max,...,capacity_integrated_Ah,capacity_provided_Ah,capacity_source,capacity_used_Ah,SOH_pct_raw,SOH_pct,EOL_cycle,RUL,capacity_initial_Ah,battery_id
0,4,0.0,6515.422,6515.422,2.907667,4.200094,3.678994,0.193375,-1.143109,0.140282,...,1.804830,1.804685,provided,1.804685,100.349508,100.000000,78,74,1.798399,B0028
1,6,0.0,6516.219,6516.219,2.892650,4.165814,3.680008,0.194487,-1.142475,-0.793146,...,1.805152,1.804783,provided,1.804783,100.354988,100.000000,78,72,1.798399,B0028
2,8,0.0,6489.500,6489.500,2.897301,4.168297,3.683014,0.192130,-1.143268,-0.792141,...,1.797954,1.797619,provided,1.797619,99.956597,99.956597,78,70,1.798399,B0028
3,10,0.0,6471.250,6471.250,2.895099,4.163485,3.683497,0.192376,-1.143568,-0.792469,...,1.793315,1.792991,provided,1.792991,99.699296,99.699296,78,68,1.798399,B0028
4,12,0.0,6466.672,6466.672,2.933874,4.164852,3.684194,0.192037,-1.144290,-0.791304,...,1.792256,1.791918,provided,1.791918,99.639612,99.639612,78,66,1.798399,B0028
5,16,0.0,6410.672,6410.672,2.888506,4.145048,3.675253,0.186895,-1.143069,-0.792867,...,1.775332,1.775026,provided,1.775026,98.700339,98.700339,78,62,1.798399,B0028
6,20,0.0,6474.156,6474.156,2.909239,4.167665,3.683659,0.187947,-1.142290,-0.790356,...,1.792356,1.792049,provided,1.792049,99.646884,99.646884,78,58,1.798399,B0028
7,23,0.0,6460.750,6460.750,2.912932,4.163373,3.684429,0.188442,-1.144330,-0.791882,...,1.790771,1.790476,provided,1.790476,99.559418,99.559418,78,55,1.798399,B0028
8,25,0.0,6448.907,6448.907,2.899377,4.165670,3.684664,0.189918,-1.143556,-0.792099,...,1.787096,1.786837,provided,1.786837,99.357055,99.357055,78,53,1.798399,B0028
9,27,0.0,6447.125,6447.125,2.914671,4.167180,3.684462,0.191013,-1.144613,-0.791842,...,1.787426,1.787153,provided,1.787153,99.374651,99.374651,78,51,1.798399,B0028


In [12]:
master_df.head(10)
master_df.tail(10)

,cycle_no,time_start,time_end,discharge_time_sec,voltage_min,voltage_max,voltage_mean,voltage_std,current_min,current_max,...,capacity_integrated_Ah,capacity_provided_Ah,capacity_source,capacity_used_Ah,SOH_pct_raw,SOH_pct,EOL_cycle,RUL,capacity_initial_Ah,battery_id
2682,229,0.0,2400.672,2400.672,2.673896,3.946639,3.334183,0.276500,-2.161573,0.167512,...,1.201118,1.198939,provided,1.198939,99.770868,99.770868,1,0,1.201692,B0056
2683,231,0.0,2396.407,2396.407,2.685663,3.946382,3.365643,0.264763,-2.160204,0.166777,...,1.148356,1.146140,provided,1.146140,95.377140,95.377140,1,0,1.201692,B0056
2684,233,0.0,2392.375,2392.375,2.674247,3.948205,3.365092,0.275905,-2.159476,0.168503,...,1.131397,1.129224,provided,1.129224,93.969499,93.969499,1,0,1.201692,B0056
2685,237,0.0,2373.687,2373.687,2.680151,3.944278,3.375040,0.265483,-2.160045,0.166308,...,1.112623,1.110343,provided,1.110343,92.398291,92.398291,1,0,1.201692,B0056
2686,239,0.0,2385.922,2385.922,2.674911,3.946104,3.344797,0.283725,-2.160992,0.167598,...,1.156116,1.153909,provided,1.153909,96.023693,96.023693,1,0,1.201692,B0056
2687,241,0.0,2345.000,2345.000,2.691357,3.948305,3.360503,0.267471,-2.160292,0.168315,...,1.132316,1.130219,provided,1.130219,94.052280,94.052280,1,0,1.201692,B0056
2688,243,0.0,2363.047,2363.047,2.675360,3.943215,3.353811,0.282247,-2.160414,0.167470,...,1.128017,1.125872,provided,1.125872,93.690523,93.690523,1,0,1.201692,B0056
2689,245,0.0,2316.687,2316.687,2.685561,3.949033,3.341671,0.274691,-2.160408,0.167226,...,1.145183,1.143011,provided,1.143011,95.116752,95.116752,1,0,1.201692,B0056
2690,249,0.0,2322.000,2322.000,2.688058,3.941156,3.345671,0.271108,-2.159901,0.168270,...,1.139442,1.137273,provided,1.137273,94.639288,94.639288,1,0,1.201692,B0056
2691,251,0.0,2357.687,2357.687,2.673133,3.943451,3.348180,0.282294,-2.161395,0.166103,...,1.132117,1.129059,provided,1.129059,93.955738,93.955738,1,0,1.201692,B0056
